In [127]:
%pip install yahooquery


In [128]:
import yahooquery as yq
from bs4 import BeautifulSoup

import datetime
import numpy as np
import pandas as pd
import requests
import seaborn as sns

sns.set_style('whitegrid')

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [129]:
#data = yq.get_currencies()
#data

In [130]:
#tickers = yq.Ticker('aapl goog')
#tickers.asset_profile

In [131]:
ativos = [
    'PETR3.SA',
    'AAPL', #apple
    'AAL', #american airlines
    'TSLA', #tesla
    'FB', #facebook
    'AMZN', #amazon
    'NFLX', #netflix
    'ABNB', #airbnb
    'MGLU3.SA', #magazine luisa
    'LINX3.SA', #LINX
    'BTOW3.SA', #B2W
    'LWSA3.SA', #LOCAWEB
    'LAME3.SA', #LOJAS AMERICANAS
    'BBDC3.SA', #BANCO BRADESCO
    'ITUB4.SA', #Itaú Unibanco Holding S.A.
    'VIVA3.SA', #VIVARA
    'VALE3.SA', #VALE
    'PCAR3.SA' # GRUPO PÃO DE AÇUCAR
]

In [132]:
ticker = yq.Ticker(ativos[6])
ticker

## Dados Históricos

    - periods = ['1d', '5d', '7d', '60d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
    - interval = ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo']

In [133]:
start_date = datetime.date.today() - datetime.timedelta(days=59)
start_date

datetime.date(2020, 11, 17)

In [134]:
data = ticker.history(period='max', adj_timezone=False)
#data = ticker.history(start=start_date, interval='1d', adj_timezone=False)
data

close        open  ...    adjclose  splits
symbol date                                ...                    
NFLX   2002-05-23    1.196429    1.156429  ...    1.196429     0.0
       2002-05-24    1.210000    1.214286  ...    1.210000     0.0
       2002-05-28    1.157143    1.213571  ...    1.157143     0.0
       2002-05-29    1.103571    1.164286  ...    1.103571     0.0
       2002-05-30    1.071429    1.107857  ...    1.071429     0.0
...                       ...         ...  ...         ...     ...
       2021-01-11  499.100006  507.839996  ...  499.100006     0.0
       2021-01-12  494.250000  500.000000  ...  494.250000     0.0
       2021-01-13  507.790009  495.500000  ...  507.790009     0.0
       2021-01-14  500.859985  507.350006  ...  500.859985     0.0
       2021-01-15  498.209991  500.000000  ...  498.209991     0.0

[4696 rows x 7 columns]

In [135]:
data.rename(str.lower, axis='columns', inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4696 entries, ('NFLX', datetime.date(2002, 5, 23)) to ('NFLX', datetime.date(2021, 1, 15))
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   close     4696 non-null   float64
 1   open      4696 non-null   float64
 2   volume    4696 non-null   int64  
 3   low       4696 non-null   float64
 4   high      4696 non-null   float64
 5   adjclose  4696 non-null   float64
 6   splits    4696 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 467.4+ KB


In [136]:
data.reset_index(level=['symbol'], inplace=True)
data.drop(columns=['symbol'], inplace=True)
data

,close,open,volume,low,high,adjclose,splits
date,,,,,,,
2002-05-23,1.196429,1.156429,104790000,1.145714,1.242857,1.196429,0.0
2002-05-24,1.210000,1.214286,11104800,1.197143,1.225000,1.210000,0.0
2002-05-28,1.157143,1.213571,6609400,1.157143,1.232143,1.157143,0.0
2002-05-29,1.103571,1.164286,6757800,1.085714,1.164286,1.103571,0.0
2002-05-30,1.071429,1.107857,10154200,1.071429,1.107857,1.071429,0.0
...,...,...,...,...,...,...,...
2021-01-11,499.100006,507.839996,3812700,497.950012,510.730011,499.100006,0.0
2021-01-12,494.250000,500.000000,5990400,485.670013,501.089996,494.250000,0.0
2021-01-13,507.790009,495.500000,5032100,493.010010,512.349976,507.790009,0.0


In [137]:
import plotly.graph_objects as go

trace1 = {
    'x': data.index,
    'open': data.open,
    'close': data.close,
    'high': data.high,
    'low': data.low,
    'type': 'candlestick',
    'name': ativos[6],
    'showlegend': False
}
 
traces = [trace1]
layout = go.Layout()
 
fig = go.Figure(data=traces, layout=layout)
fig.show()

In [138]:
#data.date = data.date.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
#data.index = data.date
#data.drop('date', axis=1, inplace=True)
#data.info()

In [139]:
#data.dropna(axis=0, inplace=True)
#data.info()

In [140]:
periods = [14, 20, 50, 200]

In [141]:
for period in periods:
    # simple moving avg
    data[f'sma_{period}'] = data.iloc[:,4].rolling(window=period).mean()
    
    # cummulative moving avg
    data[f'cma_{period}'] = data.iloc[:,4].expanding(min_periods=period).mean()

    # exponential moving avg
    data[f'ema_{period}'] = data.iloc[:,4].ewm(span=period,adjust=False).mean()

data

,close,open,volume,low,high,adjclose,splits,sma_14,cma_14,ema_14,sma_20,cma_20,ema_20,sma_50,cma_50,ema_50,sma_200,cma_200,ema_200
date,,,,,,,,,,,,,,,,,,,
2002-05-23,1.196429,1.156429,104790000,1.145714,1.242857,1.196429,0.0,NaN,NaN,1.242857,NaN,NaN,1.242857,NaN,NaN,1.242857,NaN,NaN,1.242857
2002-05-24,1.210000,1.214286,11104800,1.197143,1.225000,1.210000,0.0,NaN,NaN,1.240476,NaN,NaN,1.241156,NaN,NaN,1.242157,NaN,NaN,1.242679
2002-05-28,1.157143,1.213571,6609400,1.157143,1.232143,1.157143,0.0,NaN,NaN,1.239365,NaN,NaN,1.240298,NaN,NaN,1.241764,NaN,NaN,1.242575
2002-05-29,1.103571,1.164286,6757800,1.085714,1.164286,1.103571,0.0,NaN,NaN,1.229354,NaN,NaN,1.233059,NaN,NaN,1.238726,NaN,NaN,1.241796
2002-05-30,1.071429,1.107857,10154200,1.071429,1.107857,1.071429,0.0,NaN,NaN,1.213155,NaN,NaN,1.221135,NaN,NaN,1.233594,NaN,NaN,1.240463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-11,499.100006,507.839996,3812700,497.950012,510.730011,499.100006,0.0,526.339284,89.507945,522.224218,526.009497,89.507945,521.586149,510.001198,89.507945,515.938777,484.07045,89.507945,478.365948
2021-01-12,494.250000,500.000000,5990400,485.670013,501.089996,494.250000,0.0,524.087141,89.595646,519.406321,525.896997,89.595646,519.634134,509.744997,89.595646,515.356472,484.73310,89.595646,478.592058
2021-01-13,507.790009,495.500000,5032100,493.010010,512.349976,507.790009,0.0,522.797852,89.685709,518.465475,525.306497,89.685709,518.940405,509.874396,89.685709,515.238570,485.40585,89.685709,478.927957


In [142]:
for i in range(len(periods)):
    for j in range(i+1, len(periods)):
        p_start = periods[i]
        p_end = periods[j]
        data[f'macd_{p_start}_{p_end}'] = data[f'ema_{p_start}'] - data[f'ema_{p_end}']

data

,close,open,volume,low,high,adjclose,splits,sma_14,cma_14,ema_14,sma_20,cma_20,ema_20,sma_50,cma_50,ema_50,sma_200,cma_200,ema_200,macd_14_20,macd_14_50,macd_14_200,macd_20_50,macd_20_200,macd_50_200
date,,,,,,,,,,,,,,,,,,,,,,,,,
2002-05-23,1.196429,1.156429,104790000,1.145714,1.242857,1.196429,0.0,NaN,NaN,1.242857,NaN,NaN,1.242857,NaN,NaN,1.242857,NaN,NaN,1.242857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2002-05-24,1.210000,1.214286,11104800,1.197143,1.225000,1.210000,0.0,NaN,NaN,1.240476,NaN,NaN,1.241156,NaN,NaN,1.242157,NaN,NaN,1.242679,-0.000680,-0.001681,-0.002203,-0.001000,-0.001523,-0.000523
2002-05-28,1.157143,1.213571,6609400,1.157143,1.232143,1.157143,0.0,NaN,NaN,1.239365,NaN,NaN,1.240298,NaN,NaN,1.241764,NaN,NaN,1.242575,-0.000933,-0.002399,-0.003210,-0.001466,-0.002277,-0.000810
2002-05-29,1.103571,1.164286,6757800,1.085714,1.164286,1.103571,0.0,NaN,NaN,1.229354,NaN,NaN,1.233059,NaN,NaN,1.238726,NaN,NaN,1.241796,-0.003704,-0.009371,-0.012441,-0.005667,-0.008737,-0.003070
2002-05-30,1.071429,1.107857,10154200,1.071429,1.107857,1.071429,0.0,NaN,NaN,1.213155,NaN,NaN,1.221135,NaN,NaN,1.233594,NaN,NaN,1.240463,-0.007980,-0.020439,-0.027308,-0.012459,-0.019328,-0.006869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-11,499.100006,507.839996,3812700,497.950012,510.730011,499.100006,0.0,526.339284,89.507945,522.224218,526.009497,89.507945,521.586149,510.001198,89.507945,515.938777,484.07045,89.507945,478.365948,0.638069,6.285441,43.858270,5.647372,43.220201,37.572829
2021-01-12,494.250000,500.000000,5990400,485.670013,501.089996,494.250000,0.0,524.087141,89.595646,519.406321,525.896997,89.595646,519.634134,509.744997,89.595646,515.356472,484.73310,89.595646,478.592058,-0.227813,4.049850,40.814264,4.277662,41.042077,36.764414
2021-01-13,507.790009,495.500000,5032100,493.010010,512.349976,507.790009,0.0,522.797852,89.685709,518.465475,525.306497,89.685709,518.940405,509.874396,89.685709,515.238570,485.40585,89.685709,478.927957,-0.474929,3.226905,39.537518,3.701835,40.012447,36.310613


In [143]:
def rsi_n(prices, n=14):
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed >= 0].sum()/n
    down = -seed[seed < 0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1] # cause the diff is 1 shorter

        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)

    return rsi

for period in periods:
    data[f'rsi_{period}'] = rsi_n(data.close, period)
    data[f'over_b80_{period}'] = rsi_n(data.close, period) >= 80.
    data[f'over_b70_{period}'] = rsi_n(data.close, period) >= 70.
    data[f'over_s20_{period}'] = rsi_n(data.close, period) < 20.
    data[f'over_s30_{period}'] = rsi_n(data.close, period) < 30.

data

,close,open,volume,low,high,adjclose,splits,sma_14,cma_14,ema_14,sma_20,cma_20,ema_20,sma_50,cma_50,ema_50,sma_200,cma_200,ema_200,macd_14_20,macd_14_50,macd_14_200,macd_20_50,macd_20_200,macd_50_200,rsi_14,over_b80_14,over_b70_14,over_s20_14,over_s30_14,rsi_20,over_b80_20,over_b70_20,over_s20_20,over_s30_20,rsi_50,over_b80_50,over_b70_50,over_s20_50,over_s30_50,rsi_200,over_b80_200,over_b70_200,over_s20_200,over_s30_200
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2002-05-23,1.196429,1.156429,104790000,1.145714,1.242857,1.196429,0.0,NaN,NaN,1.242857,NaN,NaN,1.242857,NaN,NaN,1.242857,NaN,NaN,1.242857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.105409,False,False,False,False,34.454646,False,False,False,False,39.993440,False,False,False,False,49.159288,False,False,False,False
2002-05-24,1.210000,1.214286,11104800,1.197143,1.225000,1.210000,0.0,NaN,NaN,1.240476,NaN,NaN,1.241156,NaN,NaN,1.242157,NaN,NaN,1.242679,-0.000680,-0.001681,-0.002203,-0.001000,-0.001523,-0.000523,31.105409,False,False,False,False,34.454646,False,False,False,False,39.993440,False,False,False,False,49.159288,False,False,False,False
2002-05-28,1.157143,1.213571,6609400,1.157143,1.232143,1.157143,0.0,NaN,NaN,1.239365,NaN,NaN,1.240298,NaN,NaN,1.241764,NaN,NaN,1.242575,-0.000933,-0.002399,-0.003210,-0.001466,-0.002277,-0.000810,31.105409,False,False,False,False,34.454646,False,False,False,False,39.993440,False,False,False,False,49.159288,False,False,False,False
2002-05-29,1.103571,1.164286,6757800,1.085714,1.164286,1.103571,0.0,NaN,NaN,1.229354,NaN,NaN,1.233059,NaN,NaN,1.238726,NaN,NaN,1.241796,-0.003704,-0.009371,-0.012441,-0.005667,-0.008737,-0.003070,31.105409,False,False,False,False,34.454646,False,False,False,False,39.993440,False,False,False,False,49.159288,False,False,False,False
2002-05-30,1.071429,1.107857,10154200,1.071429,1.107857,1.071429,0.0,NaN,NaN,1.213155,NaN,NaN,1.221135,NaN,NaN,1.233594,NaN,NaN,1.240463,-0.007980,-0.020439,-0.027308,-0.012459,-0.019328,-0.006869,31.105409,False,False,False,False,34.454646,False,False,False,False,39.993440,False,False,False,False,49.159288,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-11,499.100006,507.839996,3812700,497.950012,510.730011,499.100006,0.0,526.339284,89.507945,522.224218,526.009497,89.507945,521.586149,510.001198,89.507945,515.938777,484.07045,89.507945,478.365948,0.638069,6.285441,43.858270,5.647372,43.220201,37.572829,43.684775,False,False,False,False,46.370712,False,False,False,False,49.976014,False,False,False,False,52.289768,False,False,False,False
2021-01-12,494.250000,500.000000,5990400,485.670013,501.089996,494.250000,0.0,524.087141,89.595646,519.406321,525.896997,89.595646,519.634134,509.744997,89.595646,515.356472,484.73310,89.595646,478.592058,-0.227813,4.049850,40.814264,4.277662,41.042077,36.764414,41.841672,False,False,False,False,45.004429,False,False,False,False,49.426797,False,False,False,False,52.134585,False,False,False,False
2021-01-13,507.790009,495.500000,5032100,493.010010,512.349976,507.790009,0.0,522.797852,89.685709,518.465475,525.306497,89.685709,518.940405,509.874396,89.685709,515.238570,485.40585,89.685709,478.927957,-0.474929,3.226905,39.537518,3.701835,40.012447,36.310613,48.388454,False,False,False,False,49.386844,False,False,False,False,50.962000,False,False,False,False,52.529860,False,False,False,False


In [144]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4497 entries, 2003-03-10 to 2021-01-15
Data columns (total 45 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   close         4497 non-null   float64
 1   open          4497 non-null   float64
 2   volume        4497 non-null   int64  
 3   low           4497 non-null   float64
 4   high          4497 non-null   float64
 5   adjclose      4497 non-null   float64
 6   splits        4497 non-null   float64
 7   sma_14        4497 non-null   float64
 8   cma_14        4497 non-null   float64
 9   ema_14        4497 non-null   float64
 10  sma_20        4497 non-null   float64
 11  cma_20        4497 non-null   float64
 12  ema_20        4497 non-null   float64
 13  sma_50        4497 non-null   float64
 14  cma_50        4497 non-null   float64
 15  ema_50        4497 non-null   float64
 16  sma_200       4497 non-null   float64
 17  cma_200       4497 non-null   float64
 18  ema_200       4497

In [145]:
X = data.replace({False: 0, True: 1})
X

,close,open,volume,low,high,adjclose,splits,sma_14,cma_14,ema_14,sma_20,cma_20,ema_20,sma_50,cma_50,ema_50,sma_200,cma_200,ema_200,macd_14_20,macd_14_50,macd_14_200,macd_20_50,macd_20_200,macd_50_200,rsi_14,over_b80_14,over_b70_14,over_s20_14,over_s30_14,rsi_20,over_b80_20,over_b70_20,over_s20_20,over_s30_20,rsi_50,over_b80_50,over_b70_50,over_s20_50,over_s30_50,rsi_200,over_b80_200,over_b70_200,over_s20_200,over_s30_200
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2003-03-10,1.123571,1.141429,3424400,1.122143,1.164286,1.123571,0.0,1.140918,0.904386,1.130939,1.090250,0.904386,1.098997,0.953371,0.904386,0.990943,0.904386,0.904386,0.941172,0.031942,0.139996,0.189766,0.108053,0.157824,0.049771,58.984983,0,0,0,0,59.833727,0,0,0,0,56.757233,0,0,0,0,49.159288,0,0,0,0
2003-03-11,1.107143,1.123571,5531400,1.071429,1.125000,1.107143,0.0,1.143469,0.905483,1.130147,1.100893,0.905483,1.101473,0.961586,0.905483,0.996200,0.903796,0.905483,0.943002,0.028674,0.133947,0.187145,0.105273,0.158471,0.053199,56.877289,0,0,0,0,58.314904,0,0,0,0,56.185975,0,0,0,0,49.046489,0,0,0,0
2003-03-12,1.075714,1.103571,5751200,1.064286,1.107857,1.075714,0.0,1.142245,0.906485,1.127175,1.109500,0.906485,1.102081,0.969571,0.906485,1.000579,0.903211,0.906485,0.944642,0.025094,0.126596,0.182533,0.101502,0.157439,0.055937,52.977285,0,0,0,0,55.479007,0,0,0,0,55.103326,0,0,0,0,48.831064,0,0,0,0
2003-03-13,1.160714,1.097143,7560000,1.089286,1.183571,1.160714,0.0,1.148214,0.907850,1.134694,1.120821,0.907850,1.109842,0.978957,0.907850,1.007755,0.902968,0.907850,0.947019,0.024852,0.126939,0.187675,0.102087,0.162823,0.060736,60.805014,0,0,0,0,60.893203,0,0,0,0,57.370269,0,0,0,0,49.434750,0,0,0,0
2003-03-14,1.182143,1.185714,6129200,1.170714,1.210714,1.182143,0.0,1.160255,0.909335,1.144830,1.133821,0.909335,1.119449,0.987171,0.909335,1.015714,0.903200,0.909335,0.949643,0.025381,0.129116,0.195187,0.103735,0.169806,0.066071,62.499821,0,0,0,0,62.115786,0,0,0,0,57.916885,0,0,0,0,49.585450,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-11,499.100006,507.839996,3812700,497.950012,510.730011,499.100006,0.0,526.339284,89.507945,522.224218,526.009497,89.507945,521.586149,510.001198,89.507945,515.938777,484.070450,89.507945,478.365948,0.638069,6.285441,43.858270,5.647372,43.220201,37.572829,43.684775,0,0,0,0,46.370712,0,0,0,0,49.976014,0,0,0,0,52.289768,0,0,0,0
2021-01-12,494.250000,500.000000,5990400,485.670013,501.089996,494.250000,0.0,524.087141,89.595646,519.406321,525.896997,89.595646,519.634134,509.744997,89.595646,515.356472,484.733100,89.595646,478.592058,-0.227813,4.049850,40.814264,4.277662,41.042077,36.764414,41.841672,0,0,0,0,45.004429,0,0,0,0,49.426797,0,0,0,0,52.134585,0,0,0,0
2021-01-13,507.790009,495.500000,5032100,493.010010,512.349976,507.790009,0.0,522.797852,89.685709,518.465475,525.306497,89.685709,518.940405,509.874396,89.685709,515.238570,485.405850,89.685709,478.927957,-0.474929,3.226905,39.537518,3.701835,40.012447,36.310613,48.388454,0,0,0,0,49.386844,0,0,0,0,50.962000,0,0,0,0,52.529860,0,0,0,0


In [146]:
a = 100
b = 98

(a / b)

1.0204081632653061

In [147]:
data.close.shift(-1)

date
2003-03-10      1.107143
2003-03-11      1.075714
2003-03-12      1.160714
2003-03-13      1.182143
2003-03-14      1.285714
                 ...    
2021-01-11    494.250000
2021-01-12    507.790009
2021-01-13    500.859985
2021-01-14    498.209991
2021-01-15           NaN
Name: close, Length: 4497, dtype: float64

In [148]:
data.close

date
2003-03-10      1.123571
2003-03-11      1.107143
2003-03-12      1.075714
2003-03-13      1.160714
2003-03-14      1.182143
                 ...    
2021-01-11    499.100006
2021-01-12    494.250000
2021-01-13    507.790009
2021-01-14    500.859985
2021-01-15    498.209991
Name: close, Length: 4497, dtype: float64

In [149]:
y = (data.close.shift(-1) / data.close) - 1.
y = (y > 0.002)
y = y.astype(int)
y

date
2003-03-10    0
2003-03-11    0
2003-03-12    1
2003-03-13    1
2003-03-14    1
             ..
2021-01-11    0
2021-01-12    1
2021-01-13    0
2021-01-14    0
2021-01-15    0
Name: close, Length: 4497, dtype: int64

In [150]:
#y = data.close.diff().shift(-1) > 0.
#y = y.astype(int)
#y

In [151]:
test_size = 73
train_size = 2485 - test_size

In [152]:
X.shape

(4497, 45)

In [153]:
train_idx = -(train_size+test_size)
test_idx = -test_size

X_train, X_test = X.values[train_idx:test_idx,:], X.values[test_idx:-1,:]
y_train, y_test = y.values[train_idx:test_idx], y.values[test_idx:-1]

In [154]:
X_train.shape, X_test.shape

((2412, 45), (72, 45))

In [155]:
y_train.shape, y_test.shape

((2412,), (72,))

In [156]:
metrics = pd.DataFrame(columns=[
    'train_acc',
    'train_f1',
    'train_prec',
    'train_rec',
    'test_acc',
    'test_f1',
    'test_prec',
    'test_rec'
])
metrics

,train_acc,train_f1,train_prec,train_rec,test_acc,test_f1,test_prec,test_rec


In [157]:
train_pred = pd.DataFrame()
test_pred = pd.DataFrame()

In [158]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

scaler = StandardScaler()

#X_train_scaled = scaler.fit_transform(X_train, y_train)
#X_test_scaled = scaler.fit_transform(X_test, y_test) 
X_train_scaled = X_train
X_test_scaled = X_test

for k in range(2, X_train.shape[1], 2):
    for n in range(1, 11, 2):
        model = KNeighborsClassifier(
            n_neighbors=n,
            weights='distance',
            leaf_size=5,
            n_jobs=-1
        )
        
        select = SelectKBest(f_classif, k=k) 
        select.fit(scaler.fit_transform(X_train_scaled), y_train)

        model.fit(select.transform(X_train_scaled), y_train)
        y_train_pred = model.predict(select.transform(X_train_scaled))
        y_pred = model.predict(select.transform(X_test_scaled))

        train_pred[f'knn{n}_k{k}'] = y_train_pred
        test_pred[f'knn{n}_k{k}'] = y_pred
        
        scores = pd.Series(data={
                'train_acc': accuracy_score(y_train_pred, y_train),
                'train_f1':f1_score(y_train_pred, y_train, average='weighted'),
                'train_prec': precision_score(y_train_pred, y_train, average='weighted'),
                'train_rec': recall_score(y_train_pred, y_train, average='weighted'),
                'test_acc': accuracy_score(y_test, y_pred),
                'test_f1': f1_score(y_test, y_pred, average='weighted'),
                'test_prec': precision_score(y_test, y_pred, average='weighted'),
                'test_rec': recall_score(y_test, y_pred, average='weighted')
            },
           name=f'knn{n}_k{k}'
        )
        
        metrics= metrics.append(scores, ignore_index=False)
        
metrics

,train_acc,train_f1,train_prec,train_rec,test_acc,test_f1,test_prec,test_rec
knn1_k2,1.0,1.0,1.0,1.0,0.472222,0.473037,0.476770,0.472222
knn3_k2,1.0,1.0,1.0,1.0,0.513889,0.513226,0.512760,0.513889
knn5_k2,1.0,1.0,1.0,1.0,0.500000,0.498438,0.497640,0.500000
knn7_k2,1.0,1.0,1.0,1.0,0.486111,0.486609,0.487294,0.486111
knn9_k2,1.0,1.0,1.0,1.0,0.472222,0.473039,0.474582,0.472222
...,...,...,...,...,...,...,...,...
knn1_k44,1.0,1.0,1.0,1.0,0.472222,0.472222,0.478827,0.472222
knn3_k44,1.0,1.0,1.0,1.0,0.486111,0.487005,0.489583,0.486111
knn5_k44,1.0,1.0,1.0,1.0,0.500000,0.500774,0.502317,0.500000
knn7_k44,1.0,1.0,1.0,1.0,0.527778,0.527778,0.527778,0.527778


In [159]:
metrics[metrics.test_acc == metrics.test_acc.max()]

,train_acc,train_f1,train_prec,train_rec,test_acc,test_f1,test_prec,test_rec
knn7_k4,1.0,1.0,1.0,1.0,0.611111,0.602604,0.643546,0.611111


In [160]:
test_balance = pd.DataFrame(index=metrics.index)
test_balance

""
knn1_k2
knn3_k2
knn5_k2
knn7_k2
knn9_k2
...
knn1_k44
knn3_k44
knn5_k44
knn7_k44


In [161]:
diff = data.close.diff().shift(-1)

diff_train, diff_test = diff.values[train_idx:test_idx], diff.values[test_idx:-1]

In [162]:
bg = []
bl = []

sg = []
sl = []

for column in test_pred.columns:
    
    bg.append(diff_test[(y_test == test_pred[column]) & (y_test == 1)].sum())
    bl.append(diff_test[(y_test != test_pred[column]) & (y_test == 1)].sum() * -1)
    
    sg.append(np.abs(diff_test[(y_test == test_pred[column]) & (y_test == 0)].sum()))
    sl.append(diff_test[(y_test != test_pred[column]) & (y_test == 0)].sum())
    
test_balance['buy_gains'] = bg
test_balance['buy_loses'] = bl
test_balance['buy_balance'] = test_balance['buy_gains'] + test_balance['buy_loses']
test_balance['sell_gains'] = sg
test_balance['sell_loses'] = sl
test_balance['sell_balance'] = test_balance['sell_gains'] + test_balance['sell_loses']
test_balance['total_balance'] = test_balance['buy_balance'] + test_balance['sell_balance']

In [163]:
test_balance

,buy_gains,buy_loses,buy_balance,sell_gains,sell_loses,sell_balance,total_balance
knn1_k2,147.030029,-155.789978,-8.759949,121.919861,-185.750153,-63.830292,-72.590240
knn3_k2,126.409973,-176.410034,-50.000061,147.499908,-160.170105,-12.670197,-62.670258
knn5_k2,118.919983,-183.900024,-64.980042,165.739899,-141.930115,23.809784,-41.170258
knn7_k2,126.409973,-176.410034,-50.000061,151.549896,-156.120117,-4.570221,-54.570282
knn9_k2,126.409973,-176.410034,-50.000061,138.909882,-168.760132,-29.850250,-79.850311
...,...,...,...,...,...,...,...
knn1_k44,158.910004,-143.910004,15.000000,122.199951,-185.470062,-63.270111,-48.270111
knn3_k44,142.770050,-160.049957,-17.279907,152.919983,-154.750031,-1.830048,-19.109955
knn5_k44,142.770050,-160.049957,-17.279907,151.950012,-155.720001,-3.769989,-21.049896
knn7_k44,142.770050,-160.049957,-17.279907,163.589996,-144.080017,19.509979,2.230072


In [164]:
test_balance.total_balance.describe()['75%']

2.230072021484375

In [165]:
best_models = test_balance[test_balance.total_balance >= test_balance.total_balance.describe()['75%']]
best_models.sort_values(by='total_balance', ascending=False)

,buy_gains,buy_loses,buy_balance,sell_gains,sell_loses,sell_balance,total_balance
knn7_k4,223.900055,-78.919952,144.980103,204.690063,-102.979950,101.710114,246.690216
knn5_k14,173.970093,-128.849915,45.120178,245.360016,-62.309998,183.050018,228.170197
knn1_k4,180.149963,-122.670044,57.479919,229.460022,-78.209991,151.250031,208.729950
knn5_k4,175.540009,-127.279999,48.260010,230.860077,-76.809937,154.050140,202.310150
knn3_k14,153.760040,-149.059967,4.700073,248.199982,-59.470032,188.729950,193.430023
knn7_k14,144.910126,-157.909882,-12.999756,246.399994,-61.270020,185.129974,172.130219
knn3_k4,174.730011,-128.089996,46.640015,215.580048,-92.089966,123.490082,170.130096
knn1_k14,165.500000,-137.320007,28.179993,208.029999,-99.640015,108.389984,136.569977
knn9_k4,218.850067,-83.969940,134.880127,154.270050,-153.399963,0.870087,135.750214
knn9_k14,144.810089,-158.009918,-13.199829,224.790009,-82.880005,141.910004,128.710175


In [165]:
model = KNeighborsClassifier(
    n_neighbors=7,
    weights='distance',
    leaf_size=5,
    n_jobs=-1
)
        
select = SelectKBest(f_classif, k=4) 
select.fit(scaler.fit_transform(X_train_scaled), y_train)

model.fit(select.transform(X_train_scaled), y_train)
model.predit()

# Features
- Inserir Low, Close, Open e High dos 3 últimos períodos
- Inserir o "tamanho" dos 3 últimos períodos
- Combinar num único valo o "tamanho" dos 3 últimos períodos

# Indicadores

- AVG Volume
- AVG Open
- AVG Close
- AVG High
- AVG Low
- RESISTENCE
- SUPPORT
- TOPO
- FUNDO
- FOUR PRICE DOJI
- GRAVESTONE DOJI
- LONG-LEGGED DOJI
- DRAGONFLY DOJI
- SPINNING TOP
- UMBRELLAS
- LONG DAY
- SHORT DAY
- MORUBOZU
- MÉDIA MÓVEL ARITMÉTICA (15, 30, 60, 90, 120)
- MÉDIA MÓVEL EXPONENCIAL 
- IFR (Índice de força relativa)
